# Data Description: Instacart Dataset

[Instacart](https://www.instacart.com/) is an online grocery retailer that sells and deliver grocery products. The data that we will use today comes from the an open sourced subset of 3 million Instacart orders and contains data on what was ordered and which customer ordered it.


In [ ]:
import pandas as pd

## Data files

The dataset was released as a part of a Kaggle competition. Instacart described the dataset in the competition description by saying,

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order. The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order. We also provide the week and hour of day the order was placed, and a relative measure of time between orders. For more information, see the blog post accompanying its [public release](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2).

“The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.kaggle.com/c/instacart-market-basket-analysis/data on 2020-12-27

### Aisles: `aisles.csv`

This file contains metadata about the aisles:

* `aisle_id`: The aisle identifier
* `aisle`: A string name describing the aisle

In [ ]:
aisle = pd.read_csv("https://compsosci-resources.s3.amazonaws.com/instacart/aisles.csv")
aisle.to_parquet("aisles.parquet")


In [ ]:
aisle.tail()

### Departments: `department.csv`

This file contains metadata about the departments:

* `department_id`: The department identifier
* `department`: A string name describing the department

In [ ]:
department = pd.read_csv("https://compsosci-resources.s3.amazonaws.com/instacart/departments.csv")
department.to_parquet("departments.parquet")

In [ ]:
department.tail()

### Products: `products.csv`

This file contains metadata about each of the products:

* `product_id`: An identifier for the product that was purchased
* `product_name`: A string name for the product's description
* `aisle_id`: The aisle identifier for the "aisle" that the item is from
* `department_id`: The department identifier for the department that the item is from

In [ ]:
products = pd.read_csv("https://compsosci-resources.s3.amazonaws.com/instacart/products.csv")
products.to_parquet("products.parquet")

In [ ]:
products.head()

In [ ]:
# Count number of items with a particular aisle/department
products.groupby(
    ["aisle_id", "department_id"]
)["product_name"].count().sort_values(
    ascending=False
).reset_index().merge(
    aisle, on="aisle_id", how="left"
).merge(
    department, on="department_id", how="left"
).sort_values(
    ["aisle_id", "department_id"]
).head(25)


### Orders: `orders.csv`

This file contains meta information about each of the 3 million orders that are covered in the dataset.

* `order_id`: A unique identifier for each order made
* `user_id`: A unique identifier for each consumer that made one of the 3 million orders
* `eval_set`: Instacart released this data for machine learning and classified orders into `prior`, `train`, and `test`
* `order_number`: The order in which the individual made the given orders
* `order_dow`: An integer between 0 and 6 denoting the day of the week the order was made
* `order_hour_of_day`: An integer between 0 and 23 denoting the hour of the day that the order was made
* `days_since_prior_order`: An integer that represents how many days it has been since a customers previous order


**What is not included?**

Instacart does not disclose the exact date of the transactions or the geography in which the transaction took place.


In [ ]:
orders = pd.read_csv("https://compsosci-resources.s3.amazonaws.com/instacart/orders.csv")
orders.to_parquet("orders.parquet")

In [ ]:
orders.describe()

In [ ]:
orders.head()

### Order Products: `order_products__prior.csv` and `order_products__train.csv`

This file contains detailed information about each of the orders:

* `order_id`: A unique identifier that describes which order the purchase belonged to
* `product_id`: An identifier for the product that was purchased
* `add_to_cart_order`: The order in which the items were placed in the cart
* `reordered`: Was the item reordered


**What is not included?**

Instacart does not disclose two pieces of information that would typically be important:

1. Price paid for a product
2. Quantity purchased

In [ ]:
prior_orders = pd.read_csv("https://compsosci-resources.s3.amazonaws.com/instacart/order_products__prior.csv")

prior_orders.describe()

In [ ]:
train_orders = pd.read_csv("https://compsosci-resources.s3.amazonaws.com/instacart/order_products__train.csv")

train_orders.describe()

In [ ]:
order_products = pd.concat([prior_orders, train_orders], axis=0, ignore_index=True)
order_products.to_parquet("order_products_all.parquet")


## Relational nature of the files

These files are organized in a way that each one contains information that either references or is referenced by one of the other files.

* `aisles.csv` and `departments.csv` are used to provide additional information and context for `products.csv`.
* `products.csv` describes the products that appear in `order_products__prior.csv` and `order_products__train.csv`.
* `orders.csv` contains information about when and who made the orders that show up in `order_products__prior.csv` and `order_products__train.csv`.

 Let's explicitly write down these relationships to set the stage for the coming lecture about SQL.

## Most frequently reordered?

Instacart was particularly interested in determining whether certain items (or groups of items) were reordered more than others.

Let's explore what items/groups were the most reordered -- We will do this by computing the fraction of "reorders" for a particular item/group ($j$) by computing:

$$\gamma_j = \frac{\sum_i \sum_{t_i} \mathbb{1}_{\text{reordered } j}}{\sum_i \sum_{t_i} \mathbb{1}_{\text{ordered } j}}$$

In [ ]:
order_product_user = order_products.merge(
    orders.loc[:, ["order_id", "user_id", "order_number", "days_since_prior_order"]],
    how="left", on="order_id"
)

In [ ]:
order_product_user.sort_values(["user_id", "order_number", "add_to_cart_order"])

**Most reordered products**


In [ ]:
mrp = order_product_user.loc[
    ~order_product_user["days_since_prior_order"].isna(), :
].groupby(
    "product_id"
).agg(
    {"add_to_cart_order": "count", "reordered": "sum"}
).query(
    "reordered > 10"
).rename(
    columns={"add_to_cart_order": "norder", "reordered": "nreorder"}
).assign(
    frac_reorder=lambda x: x["nreorder"] / x["norder"]
).sort_values(
    "frac_reorder", ascending=False
).merge(products, on="product_id", how="left")

In [ ]:
mrp.describe()

In [ ]:
mrp.head()

In [ ]:
order_product_user.query("product_id == 43553").sort_values(["user_id", "order_number"])